
<img src="../../img/ods_stickers.jpg">

## <center> [mlcourse.ai](https://mlcourse.ai) – открытый курс OpenDataScience по машинному обучению 
    
Автор материала: Юрий Кашницкий (@yorko в Slack ODS). Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание 6 (демо). Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина</center>
<img src='../../img/wine_quality.jpg' width=30%>

**Заполните пропущенный код и ответьте на вопросы в [онлайн-форме](https://docs.google.com/forms/d/1gsNxgkd0VqidZp4lh9mnCQnJw3b0IFR1C4WBES86J40).**

In [8]:
# отключим всякие предупреждения Anaconda
import warnings

warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [3]:
data = pd.read_csv('../../data/winequality-white.csv', sep=';')

In [4]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.**

In [6]:
y = data['quality']# Ваш код здесь
X = data.iloc[:, :-1]

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3, random_state=17) # Ваш код здесь
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # Ваш код здесь
X_holdout_scaled = scaler.transform(X_holdout) # Ваш код здесь

## Линейная регрессия

**Обучите простую линейную регрессию.**

In [17]:
linreg = LinearRegression() # Ваш код здесь
linreg.fit(X_train, y_train) # Ваш код здесь

LinearRegression()

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [18]:
print(f"Mean squared error (train): {mse(linreg.predict(X_train), y_train)}")# Ваш код здесь
print(f"Mean squared error (test): {mse(linreg.predict(X_holdout), y_holdout)}") # Ваш код здесь

Mean squared error (train): 0.5580606489803575
Mean squared error (test): 0.5842473102404532


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [19]:
linreg_coef = pd.DataFrame({'coef' : linreg.coef_, 'feature' : X.columns}) # Ваш код здесь
linreg_coef.sort_values(by='coef') # Ваш код здесь

,coef,feature
7,-227.551202,density
1,-1.886874,volatile acidity
2,-0.001485,citric acid
6,0.000333,total sulfur dioxide
5,0.002439,free sulfur dioxide
10,0.105264,alcohol
3,0.108183,residual sugar
0,0.113731,fixed acidity
4,0.352051,chlorides
9,0.552495,sulphates


## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.**

In [20]:
lasso1 = Lasso(alpha=0.01, random_state=17) # Ваш код здесь
lasso1.fit(X_train, y_train) # Ваш код здесь 

Lasso(alpha=0.01, random_state=17)

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [21]:
lasso1_coef = pd.DataFrame({'coef' : lasso1.coef_, 'feature' : X.columns}) # Ваш код здесь
lasso1_coef.sort_values(by='coef') # Ваш код здесь

,coef,feature
1,-0.967909,volatile acidity
0,-0.059770,fixed acidity
6,-0.001108,total sulfur dioxide
2,0.000000,citric acid
4,-0.000000,chlorides
7,-0.000000,density
8,0.000000,pH
9,0.000000,sulphates
5,0.005223,free sulfur dioxide
3,0.022231,residual sugar


**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.**

In [28]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(cv=5, alphas=alphas, random_state=17) # Ваш код здесь
lasso_cv.fit(X_train, y_train) # Ваш код здесь

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        cv=5, random_state=17)

In [29]:
best_alpha = lasso_cv.alpha_
print(best_alpha)

lasso_ba = Lasso(alpha=best_alpha, random_state=17) 
lasso_ba.fit(X_train, y_train)  

1e-06


Lasso(alpha=1e-06, random_state=17)

**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. **<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**

In [30]:
lasso_cv_coef = pd.DataFrame({'coef' : lasso_ba.coef_, 'feature' : X.columns}) # Ваш код здесь
lasso_cv_coef.sort_values(by='coef') # Ваш код здесь

,coef,feature
7,-223.055887,density
1,-1.888451,volatile acidity
2,-0.002444,citric acid
6,0.000314,total sulfur dioxide
5,0.002466,free sulfur dioxide
3,0.106579,residual sugar
0,0.110345,fixed acidity
10,0.110440,alcohol
4,0.331071,chlorides
9,0.546349,sulphates


**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [31]:
print(f"Mean squared error (train): {mse(lasso_ba.predict(X_train), y_train)}") # Ваш код здесь
print(f"Mean squared error (test): {mse(lasso_ba.predict(X_holdout), y_holdout)}") # Ваш код здесь

Mean squared error (train): 0.5580651835170742
Mean squared error (test): 0.583498303890758


## Случайный лес

**Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.**

In [32]:
forest = RandomForestRegressor(random_state=17) # Ваш код здесь
forest.fit(X_train, y_train) # Ваш код здесь

RandomForestRegressor(random_state=17)

**<font color='red'>Вопрос 5:</font> Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [37]:
print(f"Mean squared error (train): {mse(forest.predict(X_train), y_train)}") # Ваш код здесь
print(f"Mean squared error (cv): {np.mean(cross_val_score(forest, X_train, y_train, scoring='neg_mean_squared_error'))}") # Ваш код здесь
print(f"Mean squared error (test): {mse(forest.predict(X_holdout), y_holdout)}") # Ваш код здесь

Mean squared error (train): 0.052711260210035006
Mean squared error (cv): -0.41433128807644015
Mean squared error (test): 0.37076367346938777


**Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.**

In [39]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(forest, forest_params) # Ваш код здесь
locally_best_forest.fit(X_train, y_train) # Ваш код здесь

GridSearchCV(estimator=RandomForestRegressor(random_state=17),
             param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                       20, 21, 22, 23, 24],
                         'max_features': [6, 7, 8, 9, 10, 11],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7]})

In [40]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6, 'min_samples_leaf': 1},
 0.4938417994321389)

**К сожалению, результаты  GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании *random_state*). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).**<br>
**<font color='red'>Вопрос 6:</font> Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?**

In [41]:
# Ваш код здесь
forest = RandomForestRegressor(max_depth=19, max_features=7, min_samples_leaf=1, random_state=17)
forest.fit(X_train, y_train)

RandomForestRegressor(max_depth=19, max_features=7, random_state=17)

In [42]:
print(f"Mean squared error (cv): {np.mean(cross_val_score(forest, X_train, y_train, scoring='neg_mean_squared_error'))}") # Ваш код здесь
print(f"Mean squared error (test): {mse(forest.predict(X_holdout), y_holdout)}") # Ваш код здесь

Mean squared error (cv): -0.4036112411541174
Mean squared error (test): 0.3694354934060924


**Оцените важность признаков с помощью случайного леса.**<br>
**<font color='red'>Вопрос 7:</font> Какой признак оказался главным в настроенной модели случайного леса?**

In [45]:
rf_importance = pd.DataFrame({'importance' : forest.feature_importances_, 'feature' : X.columns}) # Ваш код здесь
rf_importance.sort_values(by='importance') # Ваш код здесь

,importance,feature
9,0.059499,sulphates
2,0.061500,citric acid
0,0.061873,fixed acidity
4,0.067819,chlorides
6,0.071644,total sulfur dioxide
3,0.072529,residual sugar
8,0.072646,pH
7,0.086530,density
5,0.113509,free sulfur dioxide
1,0.117627,volatile acidity


**Сделайте выводы о качестве моделей и оценках влияния признаков на качество вина с помощью этих трех моделей.**